In [58]:
import pandas as pd
import re
import spacy

In [75]:
sp = spacy.load('en_core_web_sm')

In [41]:
x1 = pd.read_csv('../data/sigmod/X1.csv')
x2 = pd.read_csv('../data/sigmod/X2.csv')
x3 = pd.read_csv('../data/sigmod/X3.csv')
x4 = pd.read_csv('../data/sigmod/X4.csv')

In [42]:
len(x1.columns), len(x2.columns), len(x3.columns), len(x4.columns)

(16, 14, 14, 5)

In [43]:
x1.columns # --> convert it to be a validation set 

Index(['cpu_brand', 'battery_chemistry', 'ram_type', 'cpu_frequency',
       'hdd_capacity', 'dimensions_height', 'brand', 'cpu_type', 'title',
       'cpu_cache', 'battery_life', 'dimensions_depth', 'dimensions',
       'display_size', 'instance_id', 'cpu_model'],
      dtype='object')

In [44]:
x2.columns

Index(['instance_id', 'brand', 'cpu_brand', 'cpu_model', 'cpu_type',
       'cpu_frequency', 'ram_capacity', 'ram_type', 'ram_frequency',
       'hdd_capacity', 'ssd_capacity', 'weight', 'dimensions', 'title'],
      dtype='object')

In [45]:
x3.columns

Index(['instance_id', 'brand', 'cpu_brand', 'cpu_model', 'cpu_type',
       'cpu_frequency', 'ram_capacity', 'ram_type', 'ram_frequency',
       'hdd_capacity', 'ssd_capacity', 'weight', 'dimensions', 'title'],
      dtype='object')

In [46]:
x4.columns

Index(['name', 'price', 'brand', 'size', 'instance_id'], dtype='object')

In [253]:
def formatNumber(num):
    num = float(num)
    if num % 1 == 0:
        return int(num)
    else:
        return num
    
extra_brands = set(pd.read_csv('laptops.csv').Company.str.lower().unique())
screen_sizes = set(pd.read_csv('laptops.csv').Inches)
screen_sizes = [str(formatNumber(str(s).lower())) for s in screen_sizes]

def preprocess_laptop_dataset(df):
    # Alpha numeric
    irrelevant_regex = re.compile(r'[^a-z0-9.\-\s]')
    multispace_regex = re.compile(r'\s\s+') # Why it doesn't work

    for column in df.columns:
        if column == 'instance_id':
            continue
        df[column] = df[column].str.lower().str.replace(irrelevant_regex, ' ').str.replace(multispace_regex, ' ')
    
    
    def tokenize_new_tile(record):
        return [w.text for w in sp(record['new_title'])]
    
    df['new_title'] = df.title
    irrelevant_regex = re.compile(r'[^a-z0-9.\s]')
    multispace_regex = re.compile(r'\s\s+') # Why it doesn't work
    df['new_title'] = df.new_title.str.lower().str.replace(irrelevant_regex, '').str.replace(multispace_regex, ' ')
    df['new_title_tokens'] =  df.apply(tokenize_new_tile, axis=1) 

    # Count the number of nans in a certain row and remove records with more than 3 nans
    #     nans_count = df.isnull().sum(axis=1)
    #     mask = nans_count > 4
    #     print("removing {} records containing nans".format(len(df[mask])))
    #     df = df[~mask]

    # Brand assignment
    all_brands = set()
    
    all_brands.update(extra_brands)

    def assign_brand(record):
        # Search in brand first
        if record['brand'] in all_brands:
            return record['brand']
        # then in the title
        for el in all_brands:
            if el in record['title']:
                return el
        return "NNN"

    df['brand'] = df.apply(assign_brand, axis=1)

    # cpu brand
    def assign_cpu_brand(record):
        # Search in brand first
        if 'intel' in str(record['cpu_brand']) or 'intel' in str(record['title']) or \
                'intel' in str(record['cpu_model']) or 'intel' in str(record['cpu_type']):
            return 'intel'
        return 'amd'

    df['cpu_brand'] = df.apply(assign_cpu_brand, axis=1)

    # cpu model
    def assign_cpu_type(record):
        
        if record['cpu_brand'] == 'intel':
            record['cpu_model'] 
            record['cpu_type']
            
        else:
            pass
    
    def assign_screen_size(record):
        brand_tokens = record['new_title_tokens']
        arr = []
        for t in brand_tokens:
            s = t.replace('inch', '')
            s = s.replace('in', '')
            arr.append(s)
        
        for sc in screen_sizes:
            if str(sc) in arr:
                return sc
        
        else:
            return 15.6 # Some relaxation
    df['screen_size'] = df.apply(assign_screen_size, axis=1)
    
    # ram capacity
    def assign_ram_capacity(record):
        s = str(record['ram_capacity']).replace(' ', '')
        possible_vals = ['2gb', '4gb', '6gb', '8gb', '10gb', '12gb', '16gb',
                         '32gb', '64gb', '128gb', '256gb', '512gb', '2', '4',
                         '6', '8', '10', '12', '16', '32', '64', '128']
        for val in possible_vals:
            if val in s:
                return int(val.replace('gb', ''))

        s = str(record['title']).replace(' ', '')  # This will be wrong, please change
        possible_vals = ['2gb', '4gb', '6gb', '8gb', '10gb', '12gb', '16gb',
                         '32gb', '64gb', '128gb']
        for val in possible_vals:
            if val in s:
                return int(val.replace('gb', ''))

        return 0
    
    def assign_laptop_model(record):
        brand_tokens = record['new_title_tokens']
        try:
            brand_index = brand_tokens.index(str(record['brand']))
            finish_index = brand_index + 2
            should_break = False
            for i in range(2 + brand_index, 5 + brand_index, 1):
                for sc in screen_sizes:
                    if (sc in brand_tokens[i]):
                        should_break = True
                        break
                if should_break:
                    if finish_index == i:
                        finish_index -=1
                    break
                if not (brand_tokens[i].isalpha()):
                    finish_index = i 
                else:
                    break
        except:
            brand_index = -1

        if brand_index == -1:
            return None

        return ' '.join(brand_tokens[brand_index+1:finish_index+1])        

    df['model'] = df.apply(assign_laptop_model, axis=1)
    df['ram_capacity'] = df.apply(assign_ram_capacity, axis=1)

    # Unit stand. in weight
    return df

In [257]:
screen_sizes, brand_toke

[11.3,
 14.1,
 10.1,
 11.6,
 12,
 13.3,
 14,
 15.4,
 15.6,
 17.3,
 13.5,
 12.5,
 13,
 18.4,
 13.9,
 17,
 15,
 12.3]

In [255]:
preprocess_laptop_dataset(x2.copy(deep=True)).brand.isna().sum()

0

In [256]:
preprocess_laptop_dataset(x3.copy(deep=True)).head(20)

,instance_id,brand,cpu_brand,cpu_model,cpu_type,cpu_frequency,ram_capacity,ram_type,ram_frequency,hdd_capacity,ssd_capacity,weight,dimensions,title,new_title,new_title_tokens,screen_size,model
0,source15__713,hp,intel,-999,intel core i7,-999,8,-999,-999,100-149 gb,-999,-999,-999,hp elitebook 8560p intel core i7 quad 2.2ghz 1...,hp elitebook 8560p intel core i7 quad 2.2ghz 1...,"[hp, elitebook, 8560p, intel, core, i7, quad, ...",15.6,
1,source15__742,hp,intel,-999,intel core i7,-999,0,-999,-999,300-499 gb,-999,-999,-999,hp 8540w notebook pc refurbished overstock.com...,hp 8540w notebook pc refurbished overstock.com...,"[hp, 8540w, notebook, pc, refurbished, oversto...",15.6,
2,source15__101,hp,intel,-999,intel core i7,-999,0,-999,-999,750 gb,-999,-999,-999,hp elitebook 8560p intel core i7 quad core 2.2...,hp elitebook 8560p intel core i7 quad core 2.2...,"[hp, elitebook, 8560p, intel, core, i7, quad, ...",15.6,
3,source7__1731,acer,intel,-999,intel core i3,1.90 ghz,6,ddr3 sdram,-999,500 gb,20 gb,4.30 lbs,9.6 in. 13.4 in,acer aspire m5-481t-33226g52mtss 14 led ultrab...,acer aspire m5481t33226g52mtss 14 led ultraboo...,"[acer, aspire, m5481t33226g52mtss, 14, led, ul...",14.0,
4,source7__1655,acer,intel,-999,intel celeron,1 ghz,4,ddr3l sdram,-999,500 gb,-999,3.04 lbs,8.1 in. 11.4 in,acer aspire v5-132p-10194g50nss 11.6 touchscre...,acer aspire v5132p10194g50nss 11.6 touchscreen...,"[acer, aspire, v5132p10194g50nss, 11.6, touchs...",11.6,
5,source7__514,hp,intel,-999,dual-core. intel core i7,2.66 ghz,4,ddr3,-999,250 gb. 1,-999,5 lbs,9.29 in. 13.21 in,hp hewlett-packard elitebook 8440p 14 notebook...,hp hewlettpackard elitebook 8440p 14 notebook ...,"[hp, hewlettpackard, elitebook, 8440p, 14, not...",14.0,
6,source7__890,asus,intel,-999,intel core i5,1.60 ghz,8,ddr3l sdram,-999,-999,256 gb,2.60 lbs,8.8 in. 12.8 in,asus zenbook ux301la-dh51t 13.3 touchscreen ul...,asus zenbook ux301ladh51t 13.3 touchscreen ult...,"[asus, zenbook, ux301ladh51, t, 13.3, touchscr...",13.3,
7,source16__2128,hp,amd,-999,-999,-999,4,-999,-999,-999,-999,-999,-999,hp elitebook revolve 810 g2 tablet - 11.6 - co...,hp elitebook revolve 810 g2 tablet 11.6 core i...,"[hp, elitebook, revolve, 810, g2, tablet, 11.6...",11.6,
8,source1__130,acer,amd,acer aspire e5-571-588m 15.6 laptop with core ...,acer aspire e5-571-588m 15.6 laptop with core ...,acer aspire e5-571-588m 15.6 laptop with core ...,4,acer aspire e5-571-588m 15.6 laptop with core ...,-999,-999,-999,-999,-999,acer aspire e5-571-588m 15.6 laptop on sale fo...,acer aspire e5571588m 15.6 laptop on sale for ...,"[acer, aspire, e5571588, m, 15.6, laptop, on, ...",15.6,
9,source1__152,hp,amd,hp 15-f009wm 15.6 notebook pc with 4gb ram 500...,hp 15-f009wm 15.6 notebook pc with 4gb ram 500...,hp 15-f009wm 15.6 notebook pc with 4gb ram 500...,4,hp 15-f009wm 15.6 notebook pc with 4gb ram 500...,-999,-999,-999,-999,-999,hp 15-f009wm 15.6 notebook pc on sale for 229.22,hp 15f009wm 15.6 notebook pc on sale for 229.22,"[hp, 15f009wm, 15.6, notebook, pc, on, sale, f...",15.6,


In [193]:
x2_dev = preprocess_laptop_dataset(x2.copy(deep=True))

In [80]:
x2_dev['tokens'] = x2_dev.title.apply(sp)

In [81]:
x2_dev.loc[:,['tokens']]

,tokens
0,"(lenovo, thinkpad, x230, 34352jf, tablet, pc, ..."
1,"(amazon.com, acer, aspire, v7, -, 582pg-6479, ..."
2,"(amazon.com, acer, aspire, e1, -, 572, -, 6870..."
3,"(amazon.com, 15.6, hp, 15, -, f009wm, amd, dua..."
4,"(amazon.com, asus, ux31a, -, xb52, 13.3, -, in..."
...,...
338,"(lenovo, thinkpad, x230, 2320, -, 12.5, -, cor..."
339,"(lenovo, thinkpad, x230, 2325, -, 12.5, -, cor..."
340,"(lenovo, thinkpad, x230, tablet, 3438, -, 12.5..."
341,"(lenovo, thinkpad, x230, 2324, -, 12.5, -, cor..."


In [109]:
x2_dev['new_title'] = x2_dev.title
irrelevant_regex = re.compile(r'[^a-z0-9.\s]')
multispace_regex = re.compile(r'\s\s+') # Why it doesn't work
x2_dev['new_title'] = x2_dev.new_title.str.lower().str.replace(irrelevant_regex, '').str.replace(multispace_regex, ' ')

In [110]:
x2_dev['model'] = pd.Series("" * len(x2_dev))

In [187]:
def get_laptop_model(record):
    brand_tokens = [w.text for w in sp(record['new_title'])]
    
    print(record['new_title'])
    
    screen_sizes = set(pd.read_csv('laptops.csv').Inches)
    screen_sizes = [str(s).lower() for s in screen_sizes]
    
    try:
        brand_index = brand_tokens.index(record['brand'])
        finish_index = brand_index + 1
        should_break = False
        for i in range(2 + brand_index, 5 + brand_index, 1):
            for sc in screen_sizes:
                if (sc in brand_tokens[i]):
                    should_break = True
                    break
            if should_break:
                break
            if not (brand_tokens[i].isalpha()):
                finish_index = i 
            else:
                break
                
    except:
        brand_index = -1
        
    if brand_index == -1:
        return ""
    
    return ' '.join(brand_tokens[brand_index+1:finish_index+1])
    

In [189]:
x2_dev

,instance_id,brand,cpu_brand,cpu_model,cpu_type,cpu_frequency,ram_capacity,ram_type,ram_frequency,hdd_capacity,ssd_capacity,weight,dimensions,title,tokens,new_title,model
0,www.softwarecity.ca//737,lenovo,intel,i5-3320m,dual-core 2 core . core i5,2.60 ghz,4,ddr3 sdram. ddr3-1600 pc3-12800. ddr3 sdram,ddr3-1600 pc3-12800,320 gb,-999,1.80 kg,-999,lenovo thinkpad x230 34352jf tablet pc - 12.5 ...,"(lenovo, thinkpad, x230, 34352jf, tablet, pc, ...",lenovo thinkpad x230 34352jf tablet pc 12.5 in...,thinkpad x230 34352jf
1,www.isupplyhub.com//1256,acer,intel,-999,1.6 ghz intel core i5-4200u,1.6 ghz intel core i5-4200u,8,ddr3 sdram. 8 gb ddr3l sdram,-999,500 gb mechanical hard drive,-999,4.8 pounds,15.02 x 10.08 x 0.90 inches,amazon.com acer aspire v7-582pg-6479 15.6-inch...,"(amazon.com, acer, aspire, v7, -, 582pg-6479, ...",amazon.com acer aspire v7582pg6479 15.6inch to...,aspire v7582pg6479
2,www.isupplyhub.com//326,acer,intel,-999,1.6 ghz intel core i5,1.6 ghz intel core i5,4,ddr3 sdram. 4 gb ddr3-sdram,-999,500 gb mechanical hard drive,-999,5.2 pounds,15.02 x 10.08 x 1 inches,amazon.com acer aspire e1-572-6870 15.6 inch l...,"(amazon.com, acer, aspire, e1, -, 572, -, 6870...",amazon.com acer aspire e15726870 15.6 inch lap...,aspire e15726870
3,www.isupplyhub.com//821,hp,amd,-999,-999,-999,4,ddr3 sdram. 4 gb sdram ddr3,-999,500 gb,-999,4.8 pounds,15.18 x 0.89 x 10.16 inches,amazon.com 15.6 hp 15-f009wm amd dual-core e1-...,"(amazon.com, 15.6, hp, 15, -, f009wm, amd, dua...",amazon.com 15.6 hp 15f009wm amd dualcore e1210...,15f009wm
4,www.isupplyhub.com//157,asus,intel,-999,1.7 ghz core i5-3317u,1.7 ghz core i5-3317u,4,ddr3 sdram. 4 gb ddr3,-999,256 mb,-999,2.9 pounds,8.80 x 0.70 x 12.80 inches,amazon.com asus ux31a-xb52 13.3-inch ultrabook...,"(amazon.com, asus, ux31a, -, xb52, 13.3, -, in...",amazon.com asus ux31axb52 13.3inch ultrabook 1...,ux31axb52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,www.vology.com//873,lenovo,intel,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,4,4 gb ddr3 slots qty 2 empty slots 1 max ram su...,4 gb ddr3 slots qty 2 empty slots 1 max ram su...,180 gb ssd. 180 gb ssd. lenovo thinkpad x230 2...,180 gb ssd. 180 gb ssd,3.3 lbs 3.3 lbs,8.1 in. 12 in x 8.1 in x 1 in. 1 in. 12 in,lenovo thinkpad x230 2320 - 12.5 - core i5 332...,"(lenovo, thinkpad, x230, 2320, -, 12.5, -, cor...",lenovo thinkpad x230 2320 12.5 core i5 3320m w...,thinkpad x230 2320
339,www.vology.com//823,lenovo,intel,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,4,4 gb ddr3 slots qty 2 max ram supported 16 gb ...,4 gb ddr3 slots qty 2 max ram supported 16 gb ...,500 gb hdd 7200 rpm. 500 gb hdd 7200 rpm. leno...,500 gb hdd 7200 rpm. 500 gb hdd 7200 rpm,3.3 lbs 3.3 lbs,8.1 in. 12 in x 8.1 in x 1 in. 1 in. 12 in,lenovo thinkpad x230 2325 - 12.5 - core i5 332...,"(lenovo, thinkpad, x230, 2325, -, 12.5, -, cor...",lenovo thinkpad x230 2325 12.5 core i5 3320m w...,thinkpad x230 2325
340,www.vology.com//2723,lenovo,intel,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,4,form factor so dimm 204-pin technology ddr3 sd...,form factor so dimm 204-pin technology ddr3 sd...,500 gb hdd 7200 rpm. 500 gb hdd 7200 rpm. leno...,500 gb hdd 7200 rpm. 500 gb hdd 7200 rpm,4 lbs 4 lbs,9 in. 12 in x 9 in x 1.2 in. 1.2 in. 12 in,lenovo thinkpad x230 tablet 3438 - 12.5 - core...,"(lenovo, thinkpad, x230, tablet, 3438, -, 12.5...",lenovo thinkpad x230 tablet 3438 12.5 core i5 ...,thinkpad x230
341,www.vology.com//1349,lenovo,intel,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,intel core i5 3rd gen 3320m 2.6 ghz. intel cor...,4,form factor so dimm 204-pin technology ddr3 sd...,fo

In [276]:
x4 = pd.read_csv('../data/sigmod/X4.csv')
x4.set_index('instance_id', inplace=True)
x4.columns

Index(['name', 'price', 'brand', 'size'], dtype='object')

In [277]:
x4.head()

,name,price,brand,size
instance_id,,,,
altosight.com//0,Lexar 32GB 1400x 210MB/s professional XQD memó...,31990.0,LEXAR,32 GB
altosight.com//25,Sony microSDXC UHS-1 U3 128GB memóriakártya + ...,59990.0,SONY,128 GB
altosight.com//66,"SANDISK DUAL DRIVE, TYPE-C, USB 3.0, 16GB, 130...",6790.0,SANDISK,16 GB
altosight.com//68,"SanDisk Dual Drive, TYPE-C, USB 3.0, 64GB, 150...",14190.0,SANDISK,64 GB
altosight.com//94,Lexar XQD 32GB X1400 PROFESSIONAL XQD +KUPON P...,329.0,LEXAR,32 GB


In [279]:
def preprocess_laptop_dataset(df):
    # Alpha numeric
    irrelevant_regex = re.compile(r'[^a-z0-9.\-\s]')
    multispace_regex = re.compile(r'\s\s+')

    for column in df.columns:
        if column == 'instance_id':
            continue
        df[column] = df[column].str.lower().str.replace(irrelevant_regex, ' ').str.replace(multispace_regex, ' ')
    
    return df

def fill_nulls_with_none(df):
    """ Fills nulls in a dataframe with None.
        This is required for the Dedupe package to work properly.
        
        Input: - dataframe with nulls as NaN
        
        Output: - new dataframe with nulls as None
    """
    new_df = df.copy()
    for col in df.columns:
        new_df[col] = new_df[col].where(new_df[col].notnull(), None)
    return new_df

def convert_numbers_to_strings(df, cols_to_convert, remove_point_zero=True):
    """ Convert number types to strings in a dataframe.
        This is convoluted as need to keep NoneTypes as NoneTypes for what comes next!
        
        Inputs: - df -> dataframe to convert number types
                - cols_to_convert -> list of columns to convert
                - remove_point_zero -> bool to say whether you want '.0' removed from number
        
        Ouputs: - dataframe with converted number types
    """
    new_df = df.copy()
    for col in cols_to_convert:
        if remove_point_zero:
            new_df[col] = new_df[col].apply(lambda x: str(x).replace('.0','')\
                                            if not isinstance(x, type(None)) else x)
        else:
            new_df[col] = new_df[col].apply(lambda x: str(x)\
                                            if not isinstance(x, type(None)) else x)
    return new_df

In [284]:
x4.info

<bound method DataFrame.info of                                                                    name  \
instance_id                                                               
altosight.com//0      Lexar 32GB 1400x 210MB/s professional XQD memó...   
altosight.com//25     Sony microSDXC UHS-1 U3 128GB memóriakártya + ...   
altosight.com//66     SANDISK DUAL DRIVE, TYPE-C, USB 3.0, 16GB, 130...   
altosight.com//68     SanDisk Dual Drive, TYPE-C, USB 3.0, 64GB, 150...   
altosight.com//94     Lexar XQD 32GB X1400 PROFESSIONAL XQD +KUPON P...   
...                                                                 ...   
altosight.com//13815                 Pendrive LEXAR Jumpdrive S25 64 GB   
altosight.com//13832              Pendrive SANDISK Extreme GO 3.1 256GB   
altosight.com//13932        Tarjeta de Memoria SD  SONY 16GB EXPERIENCE   
altosight.com//13944    Tarjeta de Memoria SANDISK Extreme Pro SDHC 8GB   
altosight.com//13950                Tarjeta de Memoria SANDISK SDHC 

In [285]:
x4_dev = convert_numbers_to_strings(x4, ['price'])
x4_dev = preprocess_laptop_dataset(x4_dev.copy(deep=True))

In [286]:
x4_dev.head()

,name,price,brand,size
instance_id,,,,
altosight.com//0,lexar 32gb 1400x 210mb s professional xqd mem ...,31990,lexar,32 gb
altosight.com//25,sony microsdxc uhs-1 u3 128gb mem riak rtya ad...,59990,sony,128 gb
altosight.com//66,sandisk dual drive type-c usb 3.0 16gb 130 mb s,6790,sandisk,16 gb
altosight.com//68,sandisk dual drive type-c usb 3.0 64gb 150 mb s,14190,sandisk,64 gb
altosight.com//94,lexar xqd 32gb x1400 professional xqd kupon pr...,329,lexar,32 gb


In [288]:
list(x4_dev.name.sample(30))

['tesco direct lexar jumpdrive c20m 32gb usb 3.0 3.1 gen 1 type-a type-c black silver',
 'sandisk ultra plus class 10 sd memory card - 64 gb',
 'cl usb intenso cl amp eacute usb 8gb intenso flashdrive premium line 3.0 - blister aluminium mk935883071 ',
 'sony usm128gqx 128 gb ram usb flash drive',
 'pam ov karta kingston sdhc 16gb uhs-i u1 90r 45w ',
 'sandisk extreme 16 gb usb-flash-laufwerk usb 3.0 bis zu 245 mb sek',
 'lexar jumpdrive c20m 128 gb micro-usb flash drive lexj4 ljdc20m-128bbeu',
 'carte m moire sdxc sandisk ultra plus 64 go classe 10',
 'tesco direct kingston technology sdxc uhs-i u3 sda3 128gb uhs class 3 memory',
 'kingston ultimate - flash-speicherkarte - 16 gb - uhs class 1 ',
 'tesco direct lexar jumpdrive s70 16gb usb memory flash drive - burgundy small blister',
 'toshiba transmemory u202 8gb usb stick usb 2.0 aqua',
 'lexar xqd 32gb 210mb s 1400x',
 'sandisk dual drive type-c usb 3.0 16gb 130 mb s',
 'kingston carte microsd action camera uhs-i u3 pour gopro dron